<a href="https://colab.research.google.com/github/AlgoAIBoss/University-Competition-First-place-code/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, mean_absolute_error, f1_score, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score, KFold
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.neighbors  import KNeighborsClassifier
import pandas_profiling as pp
from pandas_profiling import ProfileReport


In [ ]:

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Upload Data

test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DM/Data/Grigorieva.csv')

train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DM/Full.csv')

In [ ]:
train.head(10)

,date_of_match,player1_id,player1_name,player2_id,player2_name,match_outcome,lk1,lk2,month,day,year,quarter,dayofweek,dayofweek_name,weekend,time_diff,lk1-lk2,lk1lk2,lk1_freq,lk2_freq,similar_ratings
0,2020-09-06,26852620,Hildenbrand,26850588,Hardy,0,10.8,6.1,9,6,2020,3,6,Sunday,1,0,4.7,0,1.0,0.041667,0
1,2020-09-01,26852620,Hildenbrand,26852523,Janßen,1,10.8,12.8,9,1,2020,3,1,Tuesday,0,0,-2.0,1,1.0,0.041667,0
2,2020-09-01,26852620,Hildenbrand,26700909,Hippel,0,10.8,5.6,9,1,2020,3,1,Tuesday,0,1,5.2,0,1.0,0.125000,0
3,2020-08-23,26852620,Hildenbrand,26700753,Decker,0,10.8,7.0,8,23,2020,3,6,Sunday,1,0,3.8,0,1.0,0.083333,0
4,2020-08-23,26852620,Hildenbrand,26509884,Konwitschny,1,10.8,14.5,8,23,2020,3,6,Sunday,1,1,-3.7,1,1.0,0.041667,0
5,2020-08-23,26852620,Hildenbrand,26950438,Peick,0,10.8,5.8,8,23,2020,3,6,Sunday,1,1,5.0,0,1.0,0.041667,0
6,2020-08-01,26852620,Hildenbrand,27810122,Schmidt,0,10.8,4.1,8,1,2020,3,5,Saturday,1,0,6.7,0,1.0,0.041667,0
7,2020-07-18,26852620,Hildenbrand,27001351,Mehnert,0,10.8,10.9,7,18,2020,3,5,Saturday,1,0,-0.1,1,1.0,0.041667,1
8,2020-07-11,26852620,Hildenbrand,26303027,Schrenk,0,10.8,16.1,7,11,2020,3,5,Saturday,1,0,-5.3,1,1.0,0.041667,0
9,2020-07-05,26852620,Hildenbrand,27507689,Adrom,0,10.8,13.1,7,5,2020,3,6,Sunday,1,0,-2.3,1,1.0,0.041667,0


In [ ]:
train.shape

(589, 21)

In [ ]:
test.head(10)

,date_of_match,player1_id,player1_name,player2_id,player2_name,match_outcome,lk1,lk2,month,day,year,quarter,dayofweek,dayofweek_name,weekend,time_diff,lk1-lk2,lk1lk2,lk1_freq,lk2_freq,similar_ratings
0,2020-09-20,20556878,Grigorieva,29850860,Baltes,0,9.7,3.0,9,20,2020,3,6,Sunday,1,0,6.7,0,0.073171,0.006098,0
1,2020-09-20,20556878,Grigorieva,20400862,Lettgen,1,9.7,8.9,9,20,2020,3,6,Sunday,1,1,0.8,0,0.073171,0.006098,1
2,2020-09-13,20556878,Grigorieva,20300336,Seckerdieck,0,9.7,2.1,9,13,2020,3,6,Sunday,1,0,7.6,0,0.073171,0.012195,0
3,2020-12-07,20556878,Grigorieva,29152550,Hünemeier,1,9.7,13.1,12,7,2020,4,0,Monday,0,0,-3.4,1,0.073171,0.024390,0
4,2020-06-20,20556878,Grigorieva,29650130,Bernsmann,1,9.7,10.1,6,20,2020,2,5,Saturday,1,0,-0.4,1,0.073171,0.042683,0
5,2020-08-03,20556878,Grigorieva,20301932,Sparwasser,1,9.7,21.1,8,3,2020,3,0,Monday,0,0,-11.4,1,0.073171,0.012195,0
6,2020-01-26,20556878,Grigorieva,20250199,Hirt,0,9.7,1.0,1,26,2020,1,6,Sunday,1,0,8.7,0,0.073171,0.018293,0
7,2020-01-26,20556878,Grigorieva,28200730,Van Deenen,1,9.7,15.1,1,26,2020,1,6,Sunday,1,1,-5.4,1,0.073171,0.042683,0
8,2020-12-01,20556878,Grigorieva,20400943,Barth,0,9.7,1.1,12,1,2020,4,1,Tuesday,0,0,8.6,0,0.073171,0.018293,0
9,2020-12-01,20556878,Grigorieva,20352348,Kroll,1,9.7,8.1,12,1,2020,4,1,Tuesday,0,1,1.6,0,0.073171,0.030488,0


In [ ]:
test.shape

(12, 21)



---



# CatBoost - ML algorithm




---




In [ ]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Split Train / Test


X = train.drop(['match_outcome', 'date_of_match', 'player1_name', 'player2_name', 'dayofweek_name'], axis=1)
y = train.match_outcome


X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)





In [ ]:
X_test = test.drop(['match_outcome', 'date_of_match', 'player1_name', 'player2_name', 'dayofweek_name'], axis=1)
y_test = test.match_outcome


In [ ]:
# Specify Categorical features for Catboost

print(X_train.dtypes)
cat_features_train = np.where(X_train.dtypes != np.float)[0]

cat_features_train

player1_id           int64
player2_id           int64
lk1                float64
lk2                float64
month                int64
day                  int64
year                 int64
quarter              int64
dayofweek            int64
weekend              int64
time_diff            int64
lk1-lk2            float64
lk1lk2               int64
lk1_freq           float64
lk2_freq           float64
similar_ratings      int64
dtype: object


array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 12, 15])

In [ ]:
# import CatBoost Classifier

from catboost import CatBoostClassifier, Pool, cv

In [ ]:



# Train and Test for CatBoost

train_pool = Pool(X_train, y_train, cat_features=cat_features_train)


validation_pool = Pool(X_validation, y_validation, cat_features=cat_features_train)


test_pool = Pool(X_test, cat_features=cat_features_train)

In [ ]:
# CatBoost parameters

params = {"iterations": 200,        
           "eval_metric":'Accuracy',
          'combinations_ctr' : ['FloatTargetMeanValue', 'FeatureFreq'],
          'simple_ctr':['FloatTargetMeanValue', 'FeatureFreq'],
          "random_seed":42,
          "task_type": 'GPU', 
          "border_count":32,               
          'cat_features':cat_features_train,

          }           


# Model

Cat_model = CatBoostClassifier(**params)

In [ ]:

Cat_grid = {
    'depth': [2, 3, 5, 8],
    'learning_rate': [0.01, 0.03, 0.1],
    "max_ctr_complexity":[ 8, 10, 12],
    'bagging_temperature': [0, 1, 3, 5],
    'l2_leaf_reg': [3, 5, 7, 9],
    #"border_count": [7, 10, 15, 50],
}


Cat_CV = RandomizedSearchCV(Cat_model, Cat_grid, cv = 3, scoring = ['accuracy'], refit = 'accuracy', random_state=42, n_iter=10)

Cat_CV.fit(X, y)

Streaming output truncated to the last 5000 lines.
0:	learn: 0.7704082	total: 11.3ms	remaining: 2.26s
1:	learn: 0.7704082	total: 22.2ms	remaining: 2.2s
2:	learn: 0.7193878	total: 35.3ms	remaining: 2.31s
3:	learn: 0.7678571	total: 45.5ms	remaining: 2.23s
4:	learn: 0.7704082	total: 56.7ms	remaining: 2.21s
5:	learn: 0.7678571	total: 67.6ms	remaining: 2.19s
6:	learn: 0.7653061	total: 78.6ms	remaining: 2.17s
7:	learn: 0.7755102	total: 89.6ms	remaining: 2.15s
8:	learn: 0.7831633	total: 101ms	remaining: 2.14s
9:	learn: 0.7729592	total: 112ms	remaining: 2.12s
10:	learn: 0.7729592	total: 122ms	remaining: 2.1s
11:	learn: 0.7755102	total: 133ms	remaining: 2.08s
12:	learn: 0.7729592	total: 143ms	remaining: 2.06s
13:	learn: 0.7729592	total: 154ms	remaining: 2.04s
14:	learn: 0.7780612	total: 164ms	remaining: 2.02s
15:	learn: 0.7729592	total: 176ms	remaining: 2.02s
16:	learn: 0.7755102	total: 187ms	remaining: 2.01s
17:	learn: 0.7806122	total: 199ms	remaining: 2.01s
18:	learn: 0.7755102	total: 216ms	r

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7f6669a6f710>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'bagging_temperature': [0, 1, 3, 5],
                                        'depth': [2, 3, 5, 8],
                                        'l2_leaf_reg': [3, 5, 7, 9],
                                        'learning_rate': [0.01, 0.03, 0.1],
                                        'max_ctr_complexity': [8, 10, 12]},
                   pre_dispatch='2*n_jobs', random_state=42, refit='accuracy',
                   return_train_score=False, scoring=['accuracy'], verbose=0)

In [ ]:
print(f'Best Accuracy: {Cat_CV.best_score_:.3f}\n')
print(f'Best parameter set: {Cat_CV.best_params_}\n')

Best Accuracy: 0.910

Best parameter set: {'max_ctr_complexity': 10, 'learning_rate': 0.01, 'l2_leaf_reg': 5, 'depth': 8, 'bagging_temperature': 0}



In [ ]:
# Predict
Cat_prediction = Cat_CV.predict(X_test)



print(f'Scores: {classification_report(y_test, Cat_prediction)}\n')
print(f'F1-score: {accuracy_score(y_test, Cat_prediction):.3f}')




Scores:               precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.953


In [ ]:
Nitzsche['CatBoost'] = Cat_prediction

In [ ]:
submission['CatB_Accuracy'] = '95%'



---

# Logistic Regression

---



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe = LogisticRegression()

rskf = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 2, random_state = 42)

# Create param grid.

params = [
    {
        #'lg': [LogisticRegression(random_state = 42)],   
        'solver': ['newton-cg', 'lbfgs',  'sag', 'saga'],
        'penalty': ['l1', 'l2',  'elasticnet'],
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter':[100, 250, 500, 800, 1000],
        'random_state': [42]
    }
]

# Create grid search object

LR_cv = RandomizedSearchCV(pipe, params, cv = rskf, scoring = ['accuracy'], refit = 'accuracy', n_jobs = -1, random_state=42)

# Fit on data

LR_cv.fit(X, y)


In [ ]:
# Predict
LR_prediction = LR_cv.predict(X_test)



print(f'Scores: {classification_report(y_test, LR_prediction)}\n')
print(f'F1-score: {f1_score(y_test, LR_prediction):.3f}')




Scores:               precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.91      0.97      0.94        33

    accuracy                           0.91        43
   macro avg       0.89      0.83      0.86        43
weighted avg       0.91      0.91      0.90        43


F1-score: 0.941


In [ ]:
print(f'Best Accuracy: {LR_cv.best_score_:.3f}\n')
print(f'Best parameter set: {LR_cv.best_params_}\n')

Best Accuracy: 0.807

Best parameter set: {'solver': 'newton-cg', 'random_state': 42, 'penalty': 'l2', 'max_iter': 250, 'C': 10}



In [ ]:
submission['Logistic Regression'] = LR_prediction
submission['LR_Accuracy'] = '90%'


---

# RandomForest



---



In [ ]:
pipe = RandomForestClassifier()


rskf = KFold(n_splits = 5)

# Create param grid.

params = [
    {
        #'clf': [RandomForestClassifier(random_state = 42)],
        'n_estimators': [5, 50, 100, 250, 300, 350],
        'criterion': ['gini', 'entropy'],
        'max_features': ['auto', 'sqrt', 'log2'],
        'min_samples_leaf': [1, 3, 5, 8],
        'min_samples_split': [2, 4, 8],
        'max_leaf_nodes': ['None', 50, 80, 100],
        #'oob_score':[False, True],
        'max_depth': [5, 8, 10, 15, 20, 27],
        'class_weight':['None', 'balanced', 'balanced_subsample'],
        'max_leaf_nodes': ['None', 50, 80, 100],
        'random_state': [42]
    }
]

# Create grid search object

RF_CV = RandomizedSearchCV(pipe, params, cv = rskf, scoring = ['accuracy'], refit = 'accuracy', n_jobs = -1, random_state=42)

# Fit on data

RF_CV.fit(X, y)


# Scores
print(f'Best Accuracy: {RF_CV.best_score_:.3f}\n')
print(f'Best parameter set: {RF_CV.best_params_}\n')

Best Accuracy: 0.895

Best parameter set: {'random_state': 42, 'n_estimators': 100, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_leaf_nodes': 100, 'max_features': 'sqrt', 'max_depth': 8, 'criterion': 'gini', 'class_weight': 'balanced_subsample'}



In [ ]:
# Predict
RF_prediction = RF_CV.predict(X_test)



print(f' {classification_report(y_test, RF_prediction)}\n')
print(f'F1-score: {f1_score(y_test, RF_prediction):.3f}')




               precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970


In [ ]:
print(confusion_matrix(y_test, RF_prediction))

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['Random Forest'] = RF_prediction
submission['RF_Accuracy'] = '92%'



---

# KNNeighbors

---



In [ ]:
knn_model = KNeighborsClassifier()





rskf = KFold(n_splits = 5)

# Create param grid.

params = [
    {
        # 'knn': [KNeighborsClassifier()],
        'n_neighbors': [3, 5, 8, 10, 12, 15, 17, 20],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': [1, 3, 30, 50, 80, 100],
        'p': [1, 2, 8],
    }
]

# Create grid search object

KNN_CV = GridSearchCV(knn_model, params, cv = 5, scoring = ['accuracy'], refit = 'accuracy', n_jobs = -1, verbose=True)

# Fit on data

KNN_CV.fit(X, y)


# Scores
print(f'Best Accuracy: {KNN_CV.best_score_:.3f}\n')
print(f'Best parameter set: {KNN_CV.best_params_}\n')


Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 2940 tasks      | elapsed:   26.4s


Best Accuracy: 0.584

Best parameter set: {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 20, 'p': 8, 'weights': 'uniform'}



[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:   57.4s finished


In [ ]:
# Predict
KNN_prediction = KNN_CV.predict(X_test)



print(f'Scores: {classification_report(y_test, KNN_prediction)}\n')
print(f'F1-score: {f1_score(y_test, KNN_prediction):.3f}')




Scores:               precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43


F1-score: 0.868


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_test, KNN_prediction))

[[ 0 10]
 [ 0 33]]


In [ ]:
submission['KNeighbors'] = KNN_prediction
submission['KNN_Accuracy'] = '68%'



---

# ExtraTree

---



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
pipe = ExtraTreesClassifier()


# Create param grid.

params = [
    {
        'n_estimators': [3, 5, 50, 100, 250, 300, 350, 400, 500],
        'criterion': ['gini', 'entropy'],
        'min_samples_leaf': [1, 3, 5, 8],
        'min_samples_split': [2, 4, 8, 10],
        'max_features': ['auto', 'sqrt', 'log2'],
        'class_weight':['None', 'balanced', 'balanced_subsample'],
        'oob_score':[False, True],
        'max_depth': [3, 5, 8, 10, 15],
        'max_leaf_nodes': ['None', 50, 80, 100, 130, 150],
        'random_state': [42]
    }
]

# Create grid search object

EXT_CV = RandomizedSearchCV(pipe, params, cv = 3, scoring = ['accuracy'], refit = 'accuracy', n_jobs = -1, random_state=42, n_iter=10)

# Fit on data

EXT_CV.fit(X, y)




RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                        

In [ ]:
# Scores
print(f'Best Accuracy: {EXT_CV.best_score_:.3f}\n')
print(f'Best parameter set: {EXT_CV.best_params_}\n')


Best Accuracy: 0.903

Best parameter set: {'random_state': 42, 'oob_score': False, 'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_leaf_nodes': 100, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced_subsample'}



In [ ]:
# Predict
EXT_prediction = EXT_CV.predict(X_test)



print(f' {classification_report(y_test, EXT_prediction)}\n')
print(f'F1-score: {f1_score(y_test, EXT_prediction):.3f}')
print(confusion_matrix(y_test, EXT_prediction))


               precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970
[[ 9  1]
 [ 1 32]]


In [ ]:
submission['ExtraTrees'] = EXT_prediction
submission['EXT_Accuracy'] = '93%'


In [ ]:
qw = pd.DataFrame()

qw['player2_name'] = test['player2_name']
qw['lk1'] = test['lk1']
qw['lk2'] = test['lk2']
qw['test'] = y_test
qw['predict'] = RF_prediction
qw['cat'] = Cat_prediction
qw['EXT'] = EXT_prediction
qw['LR'] = LR_prediction
qw['GBC'] = GBC_prediction
qw['XGBM'] = XGB_prediction
qw['LGBM'] = LGBM_prediction
#qw['GBC_Bag'] = GBC_Bagging_prediction
qw

,player2_name,lk1,lk2,test,predict,cat,EXT,LR,GBC,XGBM,LGBM
0,Galow,8.0,10.1,1,1,1,1,1,1,1,1
1,Seitz,8.0,18.1,1,1,1,1,1,1,1,1
2,Giza,8.0,1.9,0,0,0,0,0,0,0,0
3,Ebs,8.0,10.8,1,1,1,1,1,1,1,1
4,Adamus,8.0,16.1,1,1,1,1,1,1,1,1
5,Witt,8.0,7.1,0,0,0,0,0,0,0,0
6,Remetean,8.0,7.6,0,0,0,0,0,0,0,0
7,Eberhardt,8.0,10.2,1,1,1,1,1,1,1,1
8,Steiner,8.0,8.8,1,1,1,1,1,1,1,1
9,Krajnc,8.0,13.9,1,1,1,1,1,1,1,1




---

# Deep Learning 

---



In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
pipe = MLPClassifier()


# Create param grid.

params = [
    {
        'hidden_layer_sizes': [50, 100, 150, 200],
        'activation': ['identity',  'relu'],
        'alpha': [0.0001, 0.001, 0.01],
        'solver': ['sgd', 'adam'],
        'batch_size':['auto', 10, 20],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'max_iter':[200, 300, 400],
        'learning_rate_init': [0.001, 0.01, 0.1, 0.3],
        'random_state': [42]
    }
]

# Create grid search object

MLP_CV = RandomizedSearchCV(estimator=pipe, param_distributions=params, cv = 5, scoring = ['accuracy'], refit = 'accuracy', verbose=10, n_jobs = -1, n_iter=10, random_state=42)

# Fit on data

MLP_CV.fit(X, y)




Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...
                                         'alpha': [0.0001, 0.001, 0.01],
                                         'batch_size': ['auto', 10, 20],
                                 

In [ ]:
# Scores
print(f'Best Accuracy: {MLP_CV.best_score_:.3f}\n')
print(f'Best parameter set: {MLP_CV.best_params_}\n')

Best Accuracy: 0.638

Best parameter set: {'solver': 'sgd', 'random_state': 42, 'max_iter': 400, 'learning_rate_init': 0.01, 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'batch_size': 'auto', 'alpha': 0.01, 'activation': 'relu'}



In [ ]:
# Predict
MLP_prediction = MLP_CV.predict(X_test)



print(f' {classification_report(y_test, MLP_prediction)}\n')
#print(f'F1-score: {f1_score(y_test, MLP_prediction):.3f}')
#print(confusion_matrix(y_test, MLP_prediction))



               precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
submission['MLPClassifier'] = MLP_prediction
submission['MLP_Accuracy'] = '68%'



---

# Sklearn GradientBoosting

---



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
pipe = GradientBoostingClassifier()


# Create param grid.

params = [
    {
        'n_estimators': [ 30, 50, 90, 100, 120, 250, 300, 350],
        'loss': ['deviance', 'exponential'],
        'criterion': ['friedman_mse', 'mse', 'mae'],
        'min_samples_leaf': [1, 3, 5, 8, 10, 12, 15],
        'min_samples_split': [2,3, 4, 5, 8],
        'learning_rate':[0.01, 0.05, 0.1, 0.3, 0.5],
        'max_depth': [3, 5, 8, 10, 15, 20, 25],
        'max_features': ['auto', 'sqrt', 'log2'],
        'random_state': [42]
    }
]

# Create grid search object

GBC_CV = RandomizedSearchCV(pipe, params, cv = 5, scoring = ['accuracy'], refit = 'accuracy', verbose=1, n_jobs = -1, random_state=42, n_iter=10)

# Fit on data

GBC_CV.fit(X, y)




Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   50.6s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                    

In [ ]:
# Scores
print(f'Best Accuracy: {GBC_CV.best_score_:.3f}\n')
print(f'Best parameter set: {GBC_CV.best_params_}\n')


Best Accuracy: 0.883

Best parameter set: {'random_state': 42, 'n_estimators': 50, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'max_depth': 8, 'loss': 'exponential', 'learning_rate': 0.05, 'criterion': 'mae'}



In [ ]:
# Predict
GBC_prediction = GBC_CV.predict(X_test)



print(f' {classification_report(y_test, GBC_prediction)}\n')
print(f'F1-score: {f1_score(y_test, GBC_prediction):.3f}')
print(confusion_matrix(y_test, GBC_prediction))


               precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970
[[ 9  1]
 [ 1 32]]


In [ ]:
submission['GradientBoosting'] = GBC_prediction
submission['GBC_Accuracy'] = '90%'



---

# Decision Tree

---



In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
pipe = DecisionTreeClassifier()




# Create param grid.

params = [
    {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'min_samples_leaf': [1, 3, 5, 8, 10],
        'min_samples_split': [2, 4, 8, 10],
        'max_depth': ['None', 5, 8, 10, 15, 20, 25],
        'class_weight':['None', 'balanced', 'balanced_subsample'],
        'max_leaf_nodes': ['None', 50, 80, 100],
        'max_features':['auto', 'sqrt', 'log2'],
        #'min_weight_fraction_leaf':[np.arange(0.0, 10.0, 0.2)],
        #'min_impurity_decrease': [np.arange(0.0, 10.0, 0.2)],
        #'min_impurity_split': [0, 3, 5, 8, 10],
        'random_state': [42]
    }
]




# Create grid search object

DT_CV = RandomizedSearchCV(pipe, params, cv = 5, scoring = ['accuracy'], refit = 'accuracy', n_jobs = -1, random_state=42,)




# Fit on data

DT_CV.fit(X, y)



# Scores
print(f'Best Accuracy: {DT_CV.best_score_:.3f}\n')
print(f'Best parameter set: {DT_CV.best_params_}\n')

Best Accuracy: 0.725

Best parameter set: {'splitter': 'best', 'random_state': 42, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_leaf_nodes': 100, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'entropy', 'class_weight': 'balanced'}



In [ ]:
# Predict
DT_prediction = DT_CV.predict(X_test)



print(f' {classification_report(y_test, DT_prediction)}\n')
print(f'F1-score: {f1_score(y_test, DT_prediction):.3f}')




               precision    recall  f1-score   support

           0       0.62      0.80      0.70        10
           1       0.93      0.85      0.89        33

    accuracy                           0.84        43
   macro avg       0.77      0.82      0.79        43
weighted avg       0.86      0.84      0.84        43


F1-score: 0.889


In [ ]:
print(confusion_matrix(y_test, DT_prediction))

[[ 8  2]
 [ 5 28]]


In [ ]:
submission['DecisionTree'] = DT_prediction
submission['DT_Accuracy'] = '87%'



---

# XGBoost

---



In [ ]:
import time
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier(
    tree_method = "gpu_hist",
    random_state=42, 
    eval_metric="logloss",
    objective='binary:logistic',
    seed=42
)




# Create the grid search parameter grid and scoring funcitons
param_grid = {
    "learning_rate": [0.03, 0.01, 0.1],
    "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0, 1.2],
    "max_depth": [2, 3, 4, 8, 10],
    "n_estimators": [ 200, 300, 400, 500],
    "reg_lambda": [1, 1.5, 2],
    #'reg_alpha':[0, 1e-5, 1e-2, 0.1, 1, 100],
    "gamma": [0, 0.1, 0.3],
    'min_child_weight':np.arange(1, 8, 1, dtype=int),
    'min_child_weight':np.arange(1,6,2, dtype=int)
}



XGB_model = RandomizedSearchCV( estimator=model, param_distributions=param_grid, cv=5, scoring='accuracy',
                                  n_jobs=-1, n_iter=10, refit="accuracy", verbose=True, random_state=42)



# fit grid search
%time XGB_tunned_model = XGB_model.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


CPU times: user 1.87 s, sys: 91.2 ms, total: 1.97 s
Wall time: 1min 7s


In [ ]:
print(f'Best score: {XGB_tunned_model.best_score_}')
print(f'Best model: {XGB_tunned_model.best_params_}')

Best score: 0.8843462717058224
Best model: {'subsample': 0.6, 'reg_lambda': 1, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.6}


In [ ]:
XGB_prediction = XGB_tunned_model.predict(X_test)


print('Test Accuraccy: ', accuracy_score(y_test, XGB_prediction))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, XGB_prediction))
print('\nClassification Report:')
print(classification_report(y_test, XGB_prediction))

Test Accuraccy:  0.9534883720930233

Confusion Matrix:
[[ 9  1]
 [ 1 32]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43



In [ ]:
submission['XGBoost'] = XGB_prediction
submission['XGBM_Accuracy'] = '92%'



---

# LightGBM

---



In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
grid_model = LGBMClassifier(n_iterations=500)


gridParams = {
    'learning_rate': [0.01, 0.1, 0.03, 0.3, 0.5],
    'num_leaves': [63, 90,200, 300],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'max_depth' : [2, 3,5,6,7,8],
    'random_state' : [42], 
    'colsample_bytree' : [0.3, 0.5,0.7, 0.9, 1.0],
    'subsample' : [0.1, 0.3, 0.5, 0.7, 0.9],
    'min_split_gain' : [0.01, 0.1, 0.3, 0.5],
    'min_data_in_leaf':[15, 10, 20, 5],
    'max_bin': [150, 200, 250, 300, 320, 350],   #1
    'metric':['auc', 'accuracy'],
    'num_leaves':[31, 51, 71, 90, 100],        #1
    
    }


# Initialize a RandomizedSearchCV object using 5-fold CV-
LGBM_CV = RandomizedSearchCV(grid_model, gridParams, cv = 5, random_state=42)

# Train on training data-
LGBM_CV.fit(X, y)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_iterations=500,
                                            n_jobs=-1, num_leaves=31,
                                            objective=None, random_state=None,
                                            reg_alpha=0.0, re...
                                        'max_bin': [150, 200, 250, 300, 320,
                                                

In [ ]:
print('Best parameters found by grid search are:', LGBM_CV.best_params_)
print('Best score found by grid search is:', LGBM_CV.best_score_)

Best parameters found by grid search are: {'subsample': 0.5, 'random_state': 42, 'objective': 'binary', 'num_leaves': 90, 'min_split_gain': 0.3, 'min_data_in_leaf': 10, 'metric': 'auc', 'max_depth': 3, 'max_bin': 250, 'learning_rate': 0.03, 'colsample_bytree': 1.0, 'boosting_type': 'gbdt'}
Best score found by grid search is: 0.8642329421990439


In [ ]:
LGBM_prediction = LGBM_CV.predict(X_test)


print('Test Accuraccy: ', accuracy_score(y_test, LGBM_prediction))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, LGBM_prediction))
print('\nClassification Report:')
print(classification_report(y_test, LGBM_prediction))

Test Accuraccy:  0.9534883720930233

Confusion Matrix:
[[ 9  1]
 [ 1 32]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43



In [ ]:
submission['LightGBM'] = LGBM_prediction
submission['LGBM_Accuracy'] = '91%'



---

# Bagging

---



In [ ]:
# ML Model hyperparameters

               
LGBM_params = {'n_iterations':500, 'subsample': 0.5, 'random_state': 42, 'objective': 'binary', 'num_leaves': 90, 'min_split_gain': 0.3, 'min_data_in_leaf': 10, 'metric': 'auc', 'max_depth': 3, 'max_bin': 250, 'learning_rate': 0.03, 'colsample_bytree': 1.0, 'boosting_type': 'gbdt'}

XGBM_params = {'tree_method': "gpu_hist", 'random_state':42, 'eval_metric': "logloss", 'objective':'binary:logistic', 'subsample': 0.6, 'reg_lambda': 1, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.6}

#CatB_params = {'cat_features':cat_features_train, "task_type": 'GPU', "random_state":1, 'simple_ctr':['FloatTargetMeanValue', 'FeatureFreq'], 'combinations_ctr' : ['FloatTargetMeanValue', 'FeatureFreq'], "eval_metric":'Accuracy', "iterations": 200, 'max_ctr_complexity': 8, 'learning_rate': 0.01, 'l2_leaf_reg': 7, 'depth': 5, 'bagging_temperature': 1}

RF_params = {'random_state': 42, 'n_estimators': 100, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_leaf_nodes': 100, 'max_features': 'sqrt', 'max_depth': 8, 'criterion': 'gini', 'class_weight': 'balanced_subsample'}

#LR_params = {'solver': 'newton-cg', 'random_state': 42, 'penalty': 'l2', 'max_iter': 250, 'C': 10}

#DT_params = {'splitter': 'best', 'random_state': 42, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_leaf_nodes': 100, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'entropy', 'class_weight': 'balanced'}

#KNN_params = {'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 20, 'p': 8, 'weights': 'uniform'}

#SVC_params = {'shrinking': True, 'random_state': 42, 'max_iter': -1, 'kernel': 'rbf', 'gamma': 'auto', 'degree': 12, 'decision_function_shape': 'ovr', 'class_weight': 'balanced', 'C': 10}

GBC_params = {'random_state': 42, 'n_estimators': 90, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_features': 'auto', 'max_depth': 25, 'loss': 'deviance', 'learning_rate': 0.1, 'criterion': 'mse'}

EXT_params = {'random_state': 42, 'oob_score': False, 'n_estimators': 250, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_leaf_nodes': 100, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced_subsample'}

#MLP_params = {'solver': 'sgd', 'random_state': 42, 'max_iter': 400, 'learning_rate_init': 0.01, 'learning_rate': 'invscaling', 'hidden_layer_sizes': 100, 'batch_size': 'auto', 'alpha': 0.01, 'activation': 'relu'}

In [ ]:
# LR_params = {LR_cv.best_params_}
# RF_params = {RF_CV.best_params_}
# KNN_params = {KNN_CV.best_params_}
# SVM_params = {SVC_CV.best_params_}
# EXT_params = {EXT_CV.best_params_}
# MLP_params = {MLP_CV.best_params_}
# DT_params = {DT_CV.best_params_}
# XGBM_params = {XGB_tunned_model.best_params_}
# LGBM_params = LGBM_CV.best_params_

In [ ]:
# Bagging Models

LGBM = LGBMClassifier(**LGBM_params)

XGBM = XGBClassifier(**XGBM_params)

#CatBoost = CatBoostClassifier(**CatB_params)

RF = RandomForestClassifier(**RF_params)

#LR = LogisticRegression(**LR_params)

#KNN = KNeighborsClassifier(**KNN_params)

# SVC = SVC(**SVC_params)

GBC = GradientBoostingClassifier(**GBC_params)

EXT = ExtraTreesClassifier(**EXT_params)

#DT = DecisionTreeClassifier(**DT_params)

#MLP = MLPClassifier(**MLP_params)

In [ ]:
# LightGBM Model


from sklearn.ensemble import BaggingClassifier

LGBM_Bagging = BaggingClassifier(base_estimator=LGBM, n_estimators=13, random_state=42).fit(X, y)

LGBM_Bagging_prediction = LGBM_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, LGBM_Bagging_prediction))
print(f'{classification_report(y_test, LGBM_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, LGBM_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, LGBM_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['LGBM_Bagging'] = LGBM_Bagging_prediction
submission['LGBM_Bagging_Accuracy'] = '93%'

In [ ]:
# XGBoost Model




XGBoost_Bagging = BaggingClassifier(base_estimator=XGBM, n_estimators=13, random_state=42).fit(X, y)

XGBoost_Bagging_prediction = XGBoost_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, XGBoost_Bagging_prediction))
print(f'{classification_report(y_test, XGBoost_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, XGBoost_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, XGBoost_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['XGBM_Bagging'] = XGBoost_Bagging_prediction
submission['XGBM_Bagging_Accuracy'] = '90%'

In [ ]:
# KNN Model




KNN_Bagging = BaggingClassifier(base_estimator=KNN, n_estimators=13, random_state=42).fit(X, y)

KNN_Bagging_prediction = KNN_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, KNN_Bagging_prediction))
print(f'{classification_report(y_test, KNN_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, KNN_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, KNN_Bagging_prediction))



Test Accuraccy:  0.7674418604651163
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43


F1-score: 0.868

[[ 0 10]
 [ 0 33]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
submission['KNN_Bagging'] = KNN_Bagging_prediction
submission['KNN_Bagging_Accuracy'] = '69%'

In [ ]:
# LogisticRegression Model



LR_Bagging = BaggingClassifier(base_estimator=LR, n_estimators=13, random_state=42).fit(X, y)

LR_Bagging_prediction = LR_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, LR_Bagging_prediction))
print(f'{classification_report(y_test, LR_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, LR_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, LR_Bagging_prediction))



Test Accuraccy:  0.9069767441860465
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.91      0.97      0.94        33

    accuracy                           0.91        43
   macro avg       0.89      0.83      0.86        43
weighted avg       0.91      0.91      0.90        43


F1-score: 0.941

[[ 7  3]
 [ 1 32]]


In [ ]:
submission['LR_Bagging'] = LR_Bagging_prediction
submission['LR_Bagging_Accuracy'] = '88%'

In [ ]:
# RandomForest Model



RF_Bagging = BaggingClassifier(base_estimator=RF, n_estimators=13, random_state=42).fit(X, y)

RF_Bagging_prediction = RF_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, RF_Bagging_prediction))
print(f'{classification_report(y_test, RF_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, RF_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, RF_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['RF_Bagging'] = RF_Bagging_prediction
submission['RF_Bagging_Accuracy'] = '91%'

In [ ]:
# SVC Model



SVC_Bagging = BaggingClassifier(base_estimator=SVC, n_estimators=13, random_state=42).fit(X, y)

SVC_Bagging_prediction = SVC_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, SVC_Bagging_prediction))
print(f'{classification_report(y_test, SVC_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, SVC_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, SVC_Bagging_prediction))



Test Accuraccy:  0.7804878048780488
              precision    recall  f1-score   support

           0       1.00      0.31      0.47        52
           1       0.76      1.00      0.86       112

    accuracy                           0.78       164
   macro avg       0.88      0.65      0.67       164
weighted avg       0.83      0.78      0.74       164


F1-score: 0.862

[[ 16  36]
 [  0 112]]


In [ ]:
submission['SVC_Bagging'] = SVC_Bagging_prediction
submission['SVC_Bagging_Accuracy'] = '78%'

In [ ]:
# GradientBoosting Model



GBC_Bagging = BaggingClassifier(base_estimator=GBC, n_estimators=13, random_state=42).fit(X, y)

GBC_Bagging_prediction = GBC_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, GBC_Bagging_prediction))
print(f'{classification_report(y_test, GBC_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, GBC_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, GBC_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['GBC_Bagging'] = GBC_Bagging_prediction
submission['GBC_Bagging_Accuracy'] = '91%'

In [ ]:
# ExtraTrees Model



EXT_Bagging = BaggingClassifier(base_estimator=EXT, n_estimators=13, random_state=42).fit(X, y)

EXT_Bagging_prediction = EXT_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, EXT_Bagging_prediction))
print(f'{classification_report(y_test, EXT_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, EXT_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, EXT_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['EXT_Bagging'] = EXT_Bagging_prediction
submission['EXT_Bagging_Accuracy'] = '91%'

In [ ]:
# DecisionTree Model



DT_Bagging = BaggingClassifier(base_estimator=DT, n_estimators=13, random_state=42).fit(X, y)

DT_Bagging_prediction = DT_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, DT_Bagging_prediction))
print(f'{classification_report(y_test, DT_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, DT_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, DT_Bagging_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['DT_Bagging'] = DT_Bagging_prediction
submission['DT_Bagging_Accuracy'] = '88%'

In [ ]:
# MLP Model



MLP_Bagging = BaggingClassifier(base_estimator=MLP, n_estimators=13, random_state=42).fit(X, y)

MLP_Bagging_prediction = MLP_Bagging.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, MLP_Bagging_prediction))
print(f'{classification_report(y_test, MLP_Bagging_prediction)}\n')
print(f'F1-score: {f1_score(y_test, MLP_Bagging_prediction):.3f}\n')
print(confusion_matrix(y_test, MLP_Bagging_prediction))



Test Accuraccy:  0.7674418604651163
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43


F1-score: 0.868

[[ 0 10]
 [ 0 33]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
submission['MLP_Bagging'] = MLP_Bagging_prediction
submission['MLP_Bagging_Accuracy'] = '68%'



---

# Boosting

---



In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# LightGBM Model


LGBM_Boosting = AdaBoostClassifier(base_estimator=LGBM, n_estimators=13, random_state=42).fit(X, y)

LGBM_Boosting_prediction = LGBM_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, LGBM_Boosting_prediction))
print(f'{classification_report(y_test, LGBM_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, LGBM_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, LGBM_Boosting_prediction))



Test Accuraccy:  0.7674418604651163
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43


F1-score: 0.868

[[ 0 10]
 [ 0 33]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
submission['LGBM_Boosting'] = LGBM_Boosting_prediction
submission['LGBM_Boosting_Accuracy'] = '68%'

In [ ]:
# LogisticRegression Model



LR_Boosting = AdaBoostClassifier(base_estimator=LR, n_estimators=13, random_state=42).fit(X, y)

LR_Boosting_prediction = LR_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, LR_Boosting_prediction))
print(f'{classification_report(y_test, LR_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, LR_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, LR_Boosting_prediction))



Test Accuraccy:  0.9069767441860465
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.91      0.97      0.94        33

    accuracy                           0.91        43
   macro avg       0.89      0.83      0.86        43
weighted avg       0.91      0.91      0.90        43


F1-score: 0.941

[[ 7  3]
 [ 1 32]]


In [ ]:
submission['LR_Boosting'] = LR_Boosting_prediction
submission['LR_Boosting_Accuracy'] = '87%'

In [ ]:
# RandomForest Model



RF_Boosting = AdaBoostClassifier(base_estimator=RF, n_estimators=13, random_state=42).fit(X, y)

RF_Boosting_prediction = RF_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, RF_Boosting_prediction))
print(f'{classification_report(y_test, RF_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, RF_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, RF_Boosting_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['RF_Boosting'] = RF_Boosting_prediction
submission['RF_Boosting_Accuracy'] = '91%'

In [ ]:
# SVC Model



SVC_Boosting = AdaBoostClassifier(base_estimator=SVC, n_estimators=13, algorithm='SAMME', random_state=42).fit(X, y)

SVC_Boosting_prediction = SVC_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, SVC_Boosting_prediction))
print(f'{classification_report(y_test, SVC_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, SVC_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, SVC_Boosting_prediction))



Test Accuraccy:  0.7674418604651163
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.77      1.00      0.87        33

    accuracy                           0.77        43
   macro avg       0.38      0.50      0.43        43
weighted avg       0.59      0.77      0.67        43


F1-score: 0.868

[[ 0 10]
 [ 0 33]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
submission['SVC_Boosting'] = SVC_Boosting_prediction
submission['SVC_Boosting_Accuracy'] = '68%'

In [ ]:
# GradientBoosting Model



GBC_Boosting = AdaBoostClassifier(base_estimator=GBC, n_estimators=13, random_state=42).fit(X, y)

GBC_Boosting_prediction = GBC_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, GBC_Boosting_prediction))
print(f'{classification_report(y_test, GBC_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, GBC_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, GBC_Boosting_prediction))



Test Accuraccy:  0.9302325581395349
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.92      1.00      0.96        33

    accuracy                           0.93        43
   macro avg       0.96      0.85      0.89        43
weighted avg       0.94      0.93      0.93        43


F1-score: 0.957

[[ 7  3]
 [ 0 33]]


In [ ]:
submission['GBC_Boosting'] = GBC_Boosting_prediction
submission['GBC_Boosting_Accuracy'] = '90%'

In [ ]:
# ExtraTrees Model



EXT_Boosting = AdaBoostClassifier(base_estimator=EXT, n_estimators=13, random_state=42).fit(X, y)

EXT_Boosting_prediction = EXT_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, EXT_Boosting_prediction))
print(f'{classification_report(y_test, EXT_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, EXT_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, EXT_Boosting_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['EXT_Boosting'] = EXT_Boosting_prediction
submission['EXT_Boosting_Accuracy'] = '91%'

In [ ]:
# DecisionTree Model



DT_Boosting = AdaBoostClassifier(base_estimator=DT, n_estimators=13, random_state=42).fit(X, y)

DT_Boosting_prediction = DT_Boosting.predict(X_test)




In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, DT_Boosting_prediction))
print(f'{classification_report(y_test, DT_Boosting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, DT_Boosting_prediction):.3f}\n')
print(confusion_matrix(y_test, DT_Boosting_prediction))



Test Accuraccy:  0.8837209302325582
              precision    recall  f1-score   support

           0       0.86      0.60      0.71        10
           1       0.89      0.97      0.93        33

    accuracy                           0.88        43
   macro avg       0.87      0.78      0.82        43
weighted avg       0.88      0.88      0.88        43


F1-score: 0.928

[[ 6  4]
 [ 1 32]]


In [ ]:
submission['DT_Boosting'] = DT_Boosting_prediction
submission['DT_Boosting_Accuracy'] = '87%'



---

# Stacking

---



In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

In [ ]:
# Base Models

estimators = [
              ('LGBM', LGBMClassifier(**LGBM_params)),
              ('XGBM', XGBClassifier(**XGBM_params)),
              ('RF',   RandomForestClassifier(**RF_params)),
              ('LR',   LogisticRegression(**LR_params)),
              ('KNN',  KNeighborsClassifier(**KNN_params)),
              ('SVC',  SVC(**SVC_params)),
              ('GBC',  GradientBoostingClassifier(**GBC_params)),
              ('EXT',  ExtraTreesClassifier(**EXT_params)),
              ('DT',   DecisionTreeClassifier(**DT_params)),
              ('MLP',  MLPClassifier(**MLP_params)),

              ('LGBM_Bag', BaggingClassifier(base_estimator=LGBM, n_estimators=13, random_state=42)),
              # ('XGBM_Bag', BaggingClassifier(base_estimator=XGBM, n_estimators=13, random_state=42)),
              ('RF_Bag',   BaggingClassifier(base_estimator=RF, n_estimators=13, random_state=42)),
              ('LR_Bag',   BaggingClassifier(base_estimator=LR, n_estimators=13, random_state=42)),
              # ('KNN_Bag',  BaggingClassifier(base_estimator=KNN, n_estimators=13, random_state=42)),
              # ('SVC_Bag',  BaggingClassifier(base_estimator=SVC, n_estimators=13, random_state=42)),
              ('GBC_Bag',  BaggingClassifier(base_estimator=GBC, n_estimators=13, random_state=42)),
              ('EXT_Bag',  BaggingClassifier(base_estimator=EXT, n_estimators=13, random_state=42)),
              ('DT_Bag',   BaggingClassifier(base_estimator=DT, n_estimators=13, random_state=42)),
              # ('MLP_Bag',  BaggingClassifier(base_estimator=MLP, n_estimators=13, random_state=42)),
              
              ('LGBM_Boost', AdaBoostClassifier(base_estimator=LGBM, n_estimators=13, random_state=42)),
              #('XGBM_Boost', AdaBoostClassifier(base_estimator=XGBM, n_estimators=13, random_state=42)),
              ('RF_Boost',   AdaBoostClassifier(base_estimator=RF, n_estimators=13, random_state=42)),
              ('LR_Boost',   AdaBoostClassifier(base_estimator=LR, n_estimators=13, random_state=42)),
              #('KNN_Boost',  AdaBoostClassifier(base_estimator=KNN, n_estimators=13, random_state=42)),
              #('SVC_Boost',  AdaBoostClassifier(base_estimator=SVC, n_estimators=13, random_state=42)),
              ('GBC_Boost',  AdaBoostClassifier(base_estimator=GBC, n_estimators=13, random_state=42)),
              ('EXT_Boost',  AdaBoostClassifier(base_estimator=EXT, n_estimators=13, random_state=42)),
              ('DT_Boost',   AdaBoostClassifier(base_estimator=DT, n_estimators=13, random_state=42)),
              #('MLP_Boost',  AdaBoostClassifier(base_estimator=MLP, n_estimators=13, random_state=42)),
              
              
]

In [ ]:
# Stacking Model



Stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)

Stacking_model.fit(X, y)

Stacking_prediction = Stacking_model.predict(X_test)

In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, Stacking_prediction))
print(f'{classification_report(y_test, Stacking_prediction)}\n')
print(f'F1-score: {f1_score(y_test, Stacking_prediction):.3f}\n')
print(confusion_matrix(y_test, Stacking_prediction))



Test Accuraccy:  0.9534883720930233
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        33

    accuracy                           0.95        43
   macro avg       0.93      0.93      0.93        43
weighted avg       0.95      0.95      0.95        43


F1-score: 0.970

[[ 9  1]
 [ 1 32]]


In [ ]:
submission['Stacking'] = Stacking_prediction
submission['Stacking_Accuracy'] = '92%'

In [ ]:
# Split train data in 2 parts, training and validation
training, valid, ytraining, yvalid = train_test_split(X, y, test_size=0.5)



# Specify models
model1 = RandomForestClassifier(**RF_params)
model2 = LogisticRegression(**LR_params)
model3 = CatBoostClassifier(**CatB_params)
model4 = XGBClassifier(**XGBM_params)
model5 = LGBMClassifier(**LGBM_params)
model6 = KNeighborsClassifier(**KNN_params)
model7 = SVC(**SVC_params)
model8 = GradientBoostingClassifier(**GBC_params)
model9 = ExtraTreesClassifier(**EXT_params)
model10 = DecisionTreeClassifier(**DT_params)
model11 = MLPClassifier(**MLP_params)


# Fit models
model1.fit(training, ytraining)
model2.fit(training, ytraining)
model3.fit(training, ytraining)
model4.fit(training, ytraining)
model5.fit(training, ytraining)
model6.fit(training, ytraining)
model7.fit(training, ytraining)
model8.fit(training, ytraining)
model9.fit(training, ytraining)
model10.fit(training, ytraining)
model11.fit(training, ytraining)



# make predictions for validation
preds1 = model1.predict(valid)
preds2 = model2.predict(valid)
preds3 = model3.predict(valid)
preds4 = model4.predict(valid)
preds5 = model5.predict(valid)
preds6 = model6.predict(valid)
preds7 = model7.predict(valid)
preds8 = model8.predict(valid)
preds9 = model9.predict(valid)
preds10 = model10.predict(valid)
preds11 = model11.predict(valid)



# make predictions for test data
test_preds1 = model1.predict(X_test)
test_preds2 = model2.predict(X_test)
test_preds3 = model3.predict(X_test)
test_preds4 = model4.predict(X_test)
test_preds5 = model5.predict(X_test)
test_preds6 = model6.predict(X_test)
test_preds7 = model7.predict(X_test)
test_preds8 = model8.predict(X_test)
test_preds9 = model9.predict(X_test)
test_preds10 = model10.predict(X_test)
test_preds11 = model11.predict(X_test)


# Form a new dataset for valid and test via stacking the predictions
stacked_predictions = np.column_stack((preds1, preds2, preds3, preds4, preds5, preds6, preds7, preds8, preds9, preds10, preds11))
stacked_test_predictions = np.column_stack((test_preds1, test_preds2, test_preds3, test_preds4, test_preds5, test_preds6, test_preds7, test_preds8, test_preds9, test_preds10, test_preds11))


# specify meta model
meta_model = LogisticRegression(**LR_params)

  
# fit meta model on stacked predictions
meta_model.fit(stacked_predictions, yvalid)


# make predictions on the stacked predictions of the test data
final_predictions = meta_model.predict(stacked_test_predictions)


In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, final_predictions))
print(f'{classification_report(y_test, final_predictions)}\n')
print(f'F1-score: {f1_score(y_test, final_predictions):.3f}\n')
print(confusion_matrix(y_test, final_predictions))



Test Accuraccy:  0.8597560975609756
              precision    recall  f1-score   support

           0       0.81      0.73      0.77        52
           1       0.88      0.92      0.90       112

    accuracy                           0.86       164
   macro avg       0.84      0.83      0.83       164
weighted avg       0.86      0.86      0.86       164


F1-score: 0.900

[[ 38  14]
 [  9 103]]




---

# VotingClassifier

---



In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
# Base Models

estimators = [
              ('LGBM', LGBMClassifier(**LGBM_params)),
              ('XGBM', XGBClassifier(**XGBM_params)),
              #('CatBoost', CatBoostClassifier(**CatB_params)),
              ('RF', RandomForestClassifier(**RF_params)),
              ('LR', LogisticRegression(**LR_params)),
              ('KNN', KNeighborsClassifier(**KNN_params)),
              ('SVC', SVC(**SVC_params)),
              ('GBC', GradientBoostingClassifier(**GBC_params)),
              ('EXT', ExtraTreesClassifier(**EXT_params)),
              ('DT',  DecisionTreeClassifier(**DT_params)),
              ('MLP', MLPClassifier(**MLP_params)),
]

In [ ]:
# Voting Model



Voting_model = VotingClassifier(estimators=estimators, voting='hard',  n_jobs=-1)

Voting_model.fit(X, y)

Voting_prediction = Voting_model.predict(X_test)

In [ ]:
# Metrix

print('Test Accuraccy: ', accuracy_score(y_test, Voting_prediction))
print(f'{classification_report(y_test, Voting_prediction)}\n')
print(f'F1-score: {f1_score(y_test, Voting_prediction):.3f}\n')
print(confusion_matrix(y_test, Voting_prediction))



Test Accuraccy:  0.9207317073170732
              precision    recall  f1-score   support

           0       0.91      0.83      0.87        52
           1       0.92      0.96      0.94       112

    accuracy                           0.92       164
   macro avg       0.92      0.90      0.91       164
weighted avg       0.92      0.92      0.92       164


F1-score: 0.943

[[ 43   9]
 [  4 108]]


In [ ]:
submission['Voting'] = Voting_prediction
submission['Voting_Accuracy'] = '92%'



---

# TO DO

---



In [ ]:
#  import pywFM
#  SVM
#  ExtraTree
#  Sklearn GradientBoosting
#  VotingClassifier
#  use regression in classification
#  Check out xcessiv
#  kaz-Anova StackNet
# Learn Hyperparameter best score



---

# Conclusion

---



In [ ]:
Nitzsche = pd.DataFrame()
Nitzsche['player1_name'] = test['player1_name']
Nitzsche['player2_name'] = test['player2_name']

In [ ]:
Nitzsche['LightGBM'] = LGBM_prediction

In [ ]:
Nitzsche

,player1_name,player2_name,LightGBM
0,Nitzsche,Galow,1
1,Nitzsche,Seitz,1
2,Nitzsche,Giza,0
3,Nitzsche,Ebs,1
4,Nitzsche,Adamus,1
5,Nitzsche,Witt,0
6,Nitzsche,Remetean,0
7,Nitzsche,Eberhardt,1
8,Nitzsche,Steiner,1
9,Nitzsche,Krajnc,1


In [ ]:
Nitzsche.to_csv('/content/gdrive/MyDrive/Colab Notebooks/DM/Nitzsche_submission.csv', index=False)

In [ ]:
from prettytable import PrettyTable

x=PrettyTable(['Algorithm','Accuracy score'])
x.add_row(['CatBoost',               0.9012])
x.add_row(['------------------',  '----------'])
x.add_row(['CatBoost(Kfold)',        0.7975])
x.add_row(['------------------',  '----------'])
x.add_row(['XGBoost',                0.8636])
x.add_row(['------------------',  '----------'])
x.add_row(['XGBoost(KFold)',         0.8547])
x.add_row(['------------------',  '----------'])
x.add_row(['LightGBM',               0.8977])
x.add_row(['------------------',  '----------'])
x.add_row(['LightGBM(KFold)',        0.8507])
x.add_row(['------------------',  '----------'])
x.add_row(['Random Forest',          0.9012])
x.add_row(['------------------',  '----------'])
x.add_row(['Random Forest(KFold)',   0.9012])
x.add_row(['------------------',  '----------'])
x.add_row(['Logistic Regression',    0.9012])
x.add_row(['------------------',  '----------'])
x.add_row(['Logistic Reg(KFold)',    0.7973])
x.add_row(['------------------',  '----------'])
x.add_row(['KNN',                    0.6123])
x.add_row(['------------------',  '----------'])
x.add_row(['KNN(KFold)',             0.6601])
x.add_row(['------------------',  '----------'])
x.add_row(['SVC',                    0.6111])
x.add_row(['------------------',  '----------'])
x.add_row(['SVC(KFold)',             0.6511])
x.add_row(['------------------',  '----------'])
x.add_row(['ExtraTrees',             0.9012])
x.add_row(['------------------',  '----------'])
x.add_row(['ExtraTrees(KFold)',      0.8155])
x.add_row(['------------------',  '----------'])
x.add_row(['Deep Learning',          0.6912])
x.add_row(['------------------',  '----------'])
x.add_row(['Deep Learning(KFold)',   0.6555])
x.add_row(['------------------',  '----------'])
x.add_row(['GradientBoosting',       0.8812])
x.add_row(['------------------',  '----------'])
x.add_row(['GradientBoosting(KFold)',0.8155])
x.add_row(['------------------',  '----------'])
x.add_row(['DecisionTree',           0.8612])
x.add_row(['------------------',  '----------'])
x.add_row(['DecisionTree(KFold)',    0.7911])
print(x)

+-------------------------+----------------+
|        Algorithm        | Accuracy score |
+-------------------------+----------------+
|         CatBoost        |     0.9012     |
|    ------------------   |   ----------   |
|     CatBoost(Kfold)     |     0.7975     |
|    ------------------   |   ----------   |
|         XGBoost         |     0.8636     |
|    ------------------   |   ----------   |
|      XGBoost(KFold)     |     0.8547     |
|    ------------------   |   ----------   |
|         LightGBM        |     0.8977     |
|    ------------------   |   ----------   |
|     LightGBM(KFold)     |     0.8507     |
|    ------------------   |   ----------   |
|      Random Forest      |     0.9012     |
|    ------------------   |   ----------   |
|   Random Forest(KFold)  |     0.9012     |
|    ------------------   |   ----------   |
|   Logistic Regression   |     0.9012     |
|    ------------------   |   ----------   |
|   Logistic Reg(KFold)   |     0.7973     |
|    -----

In [ ]:
# Bagging

x=PrettyTable(['Bagging','Accuracy score'])
x.add_row(['XGBoost',                0.901])
x.add_row(['------------------',  '----------'])
x.add_row(['LightGBM',               0.901])
x.add_row(['------------------',  '----------'])
x.add_row(['Random Forest',          0.92])
x.add_row(['------------------',  '----------'])
x.add_row(['Logistic Regression',    0.901])
x.add_row(['------------------',  '----------'])
x.add_row(['KNN',                    0.65])
x.add_row(['------------------',  '----------'])
x.add_row(['SVC',                    0.61])
x.add_row(['------------------',  '----------'])
x.add_row(['ExtraTrees',             0.91])
x.add_row(['------------------',  '----------'])
x.add_row(['Deep Learning',          0.69])
x.add_row(['------------------',  '----------'])
x.add_row(['GradientBoosting',       0.91])
x.add_row(['------------------',  '----------'])
x.add_row(['DecisionTree',           0.86])
print(x)

+---------------------+----------------+
|       Bagging       | Accuracy score |
+---------------------+----------------+
|       XGBoost       |     0.901      |
|  ------------------ |   ----------   |
|       LightGBM      |     0.901      |
|  ------------------ |   ----------   |
|    Random Forest    |      0.92      |
|  ------------------ |   ----------   |
| Logistic Regression |     0.901      |
|  ------------------ |   ----------   |
|         KNN         |      0.65      |
|  ------------------ |   ----------   |
|         SVC         |      0.61      |
|  ------------------ |   ----------   |
|      ExtraTrees     |      0.91      |
|  ------------------ |   ----------   |
|    Deep Learning    |      0.69      |
|  ------------------ |   ----------   |
|   GradientBoosting  |      0.91      |
|  ------------------ |   ----------   |
|     DecisionTree    |      0.86      |
+---------------------+----------------+


In [ ]:
# Boosting

x=PrettyTable(['Bagging','Accuracy score'])
x.add_row(['LightGBM',               0.69])
x.add_row(['------------------',  '----------'])
x.add_row(['Random Forest',          0.86])
x.add_row(['------------------',  '----------'])
x.add_row(['Logistic Regression',    0.901])
x.add_row(['------------------',  '----------'])
x.add_row(['SVC',                    0.69])
x.add_row(['------------------',  '----------'])
x.add_row(['ExtraTrees',             0.65])
x.add_row(['------------------',  '----------'])
x.add_row(['GradientBoosting',       0.64])
x.add_row(['------------------',  '----------'])
x.add_row(['DecisionTree',           0.84])
print(x)

+---------------------+----------------+
|       Bagging       | Accuracy score |
+---------------------+----------------+
|       LightGBM      |      0.69      |
|  ------------------ |   ----------   |
|    Random Forest    |      0.86      |
|  ------------------ |   ----------   |
| Logistic Regression |     0.901      |
|  ------------------ |   ----------   |
|         SVC         |      0.69      |
|  ------------------ |   ----------   |
|      ExtraTrees     |      0.65      |
|  ------------------ |   ----------   |
|   GradientBoosting  |      0.64      |
|  ------------------ |   ----------   |
|     DecisionTree    |      0.84      |
+---------------------+----------------+
